In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('merged_data_step_1_dropped.csv')
df

C:\Users\Matin\AppData\Local\Temp\ipykernel_67964\2800432195.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_data_step_1_dropped.csv')


,Year,FHLBankID,FIPSStateCode,FIPSCountyCode,MSA,Tract,MinPer,TraMedY,LocMedY,Income,...,CoGender,BoAge,CoAge,Rate,Amount,Front,Back,Borrower Credit Score,Co-Borrower Credit Score,Self
0,2009,NaN,39,37,99999,5701.02,1.800,45735,43801,39312,...,4,35.0,98.0,0.0463,156200,0.31,0.31,5.0,9,2
1,2009,NaN,21,117,17140,647.00,2.430,67218,54771,75322,...,4,36.0,98.0,0.0488,148000,0.19,0.23,5.0,9,2
2,2009,NaN,21,211,31140,405.00,16.380,66717,49301,40344,...,2,42.0,38.0,0.0488,104000,0.23,0.24,5.0,5,2
3,2009,NaN,21,123,21060,9601.00,8.060,40083,42911,88440,...,2,33.0,32.0,0.0488,187200,0.18,0.35,5.0,5,2
4,2009,NaN,21,211,31140,401.00,3.780,48655,49301,91224,...,2,41.0,40.0,0.0500,136000,0.12,0.43,4.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743828,2021,Topeka,20,113,32700,7881.00,2.967,76364,71728,5936,...,4,81.0,998.0,2.8750,164900,24.38,25.38,4.0,9,2
743829,2021,Topeka,8,5,19740,832.00,36.160,110260,98577,6331,...,2,59.0,58.0,2.7500,410000,34.79,51.21,3.0,5,2
743830,2021,Topeka,8,41,17820,69.02,22.138,88309,81873,6927,...,2,37.0,29.0,2.3750,540000,36.01,45.03,5.0,5,2
743831,2021,Topeka,8,41,17820,51.06,28.995,90585,81873,12905,...,2,41.0,38.0,2.5000,354700,16.40,22.51,1.0,1,2


In [5]:
df.groupby("Year")[['FHLBankID', 'BoAge', 'CoAge']].apply(lambda x: x.isnull().sum())

,FHLBankID,BoAge,CoAge
Year,,,
2009,50502,5014,3636
2010,0,6589,4698
2011,0,6201,4234
2012,0,10051,6932
2013,0,6211,3884
2014,0,4467,2733
2015,0,5399,3371
2016,0,5554,3446
2017,0,2916,1869


In [62]:
df_droppedNA = df.dropna(subset=['BoAge', 'CoAge', 'Borrower Credit Score'])

In [ ]:
df_droppedNA.groupby("Year")[['FHLBankID', 'BoAge', 'CoAge', 'Borrower Credit Score']].apply(lambda x: x.isnull().sum())

Missing values per year for each feature:
      FHLBankID  BoAge  CoAge  Borrower Credit Score
Year                                                
2009      45376      0      0                      0
2010          0      0      0                      0
2011          0      0      0                      0
2012          0      0      0                      0
2013          0      0      0                      0
2014          0      0      0                      0
2015          0      0      0                      0
2016          0      0      0                      0
2017          0      0      0                      0
2018          0      0      0                      0
2019          0      0      0                      0
2020          0      0      0                      0
2021          0      0      0                      0


In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = [col for col in df_droppedNA.columns if col not in ['FHLBankID']]
known_data = df_droppedNA[df_droppedNA['FHLBankID'].notnull()]
X_train, X_val, y_train, y_val = train_test_split(known_data[features], known_data['FHLBankID'], test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.9193


In [65]:
X_test = df_droppedNA[df_droppedNA['FHLBankID'].isnull()][features]
df_droppedNA.loc[df_droppedNA['FHLBankID'].isnull(), 'FHLBankID'] = rf_model.predict(X_test)

In [66]:
df_droppedNA

,Year,FHLBankID,FIPSStateCode,FIPSCountyCode,MSA,Tract,MinPer,TraMedY,LocMedY,Income,...,CoGender,BoAge,CoAge,Rate,Amount,Front,Back,Borrower Credit Score,Co-Borrower Credit Score,Self
0,2009,Cincinnati,39,37,99999,5701.02,1.800,45735,43801,39312,...,4,35.0,98.0,0.0463,156200,0.31,0.31,5.0,9,2
1,2009,Cincinnati,21,117,17140,647.00,2.430,67218,54771,75322,...,4,36.0,98.0,0.0488,148000,0.19,0.23,5.0,9,2
2,2009,Cincinnati,21,211,31140,405.00,16.380,66717,49301,40344,...,2,42.0,38.0,0.0488,104000,0.23,0.24,5.0,5,2
3,2009,Cincinnati,21,123,21060,9601.00,8.060,40083,42911,88440,...,2,33.0,32.0,0.0488,187200,0.18,0.35,5.0,5,2
4,2009,Cincinnati,21,211,31140,401.00,3.780,48655,49301,91224,...,2,41.0,40.0,0.0500,136000,0.12,0.43,4.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743828,2021,Topeka,20,113,32700,7881.00,2.967,76364,71728,5936,...,4,81.0,998.0,2.8750,164900,24.38,25.38,4.0,9,2
743829,2021,Topeka,8,5,19740,832.00,36.160,110260,98577,6331,...,2,59.0,58.0,2.7500,410000,34.79,51.21,3.0,5,2
743830,2021,Topeka,8,41,17820,69.02,22.138,88309,81873,6927,...,2,37.0,29.0,2.3750,540000,36.01,45.03,5.0,5,2
743831,2021,Topeka,8,41,17820,51.06,28.995,90585,81873,12905,...,2,41.0,38.0,2.5000,354700,16.40,22.51,1.0,1,2


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_droppedNA.loc[df_droppedNA["FHLBankID"].notnull(), "FHLBankID"] = le.fit_transform(df_droppedNA.loc[df_droppedNA["FHLBankID"].notnull(), "FHLBankID"])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Encoding Mapping:")
print(label_mapping)

df_droppedNA = df_droppedNA[~((df_droppedNA['Income'] == 1) & (df_droppedNA['Year'].isin([2010, 2011])))]
df_droppedNA['CoAge'] = df_droppedNA['CoAge'].replace(98, 998)

Label Encoding Mapping:
{'Atlanta': 0, 'Boston': 1, 'Chicago': 2, 'Cincinnati': 3, 'Dallas': 4, 'Des Moines': 5, 'Indianapolis': 6, 'New York': 7, 'Pittsburgh': 8, 'San Francisco': 9, 'Topeka': 10}


In [ ]:
df_droppedNA.to_csv("merged_data_step_2_Cleaned.csv", index=False)

In [41]:
df_droppedNA

,Year,FHLBankID,FIPSStateCode,FIPSCountyCode,MSA,Tract,MinPer,TraMedY,LocMedY,Income,...,CoGender,BoAge,CoAge,Rate,Amount,Front,Back,Borrower Credit Score,Co-Borrower Credit Score,Self
0,2009,3,39,37,99999,5701.02,1.800,45735,43801,39312,...,4,35.0,998.0,0.0463,156200,0.31,0.31,5.0,9,2
1,2009,3,21,117,17140,647.00,2.430,67218,54771,75322,...,4,36.0,998.0,0.0488,148000,0.19,0.23,5.0,9,2
2,2009,3,21,211,31140,405.00,16.380,66717,49301,40344,...,2,42.0,38.0,0.0488,104000,0.23,0.24,5.0,5,2
3,2009,3,21,123,21060,9601.00,8.060,40083,42911,88440,...,2,33.0,32.0,0.0488,187200,0.18,0.35,5.0,5,2
4,2009,3,21,211,31140,401.00,3.780,48655,49301,91224,...,2,41.0,40.0,0.0500,136000,0.12,0.43,4.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743828,2021,10,20,113,32700,7881.00,2.967,76364,71728,5936,...,4,81.0,998.0,2.8750,164900,24.38,25.38,4.0,9,2
743829,2021,10,8,5,19740,832.00,36.160,110260,98577,6331,...,2,59.0,58.0,2.7500,410000,34.79,51.21,3.0,5,2
743830,2021,10,8,41,17820,69.02,22.138,88309,81873,6927,...,2,37.0,29.0,2.3750,540000,36.01,45.03,5.0,5,2
743831,2021,10,8,41,17820,51.06,28.995,90585,81873,12905,...,2,41.0,38.0,2.5000,354700,16.40,22.51,1.0,1,2


In [ ]:
import pandas as pd
df_droppedNA = pd.read_csv("merged_data_step_2_Cleaned.csv")

categorical_features = ['FHLBankID', 'Purpose', 'NumBor', 'BoRace', 'CoRace', 'BoGender', 'CoGender', 'Borrower Credit Score', 'Co-Borrower Credit Score', 'Self']  # needs to adjust 
categorical_features += ['FIPSStateCode', 'FIPSCountyCode', 'MSA']
df_encoded = pd.get_dummies(df_droppedNA, columns=categorical_features, drop_first=True)
print(df_encoded.head())
print(f"Original : {df_droppedNA.shape}, Encoded : {df_encoded.shape}")
df_encoded.to_csv("merged_data_step_3_Encoded.csv", index=False)

   Year    Tract  MinPer  TraMedY  LocMedY  Income  CurAreY     UPB   LTV  \
0  2009  5701.02    1.80    45735    43801   39312    51600  156200  0.70   
1  2009   647.00    2.43    67218    54771   75322    66200  148000  0.80   
2  2009   405.00   16.38    66717    49301   40344    59400  104000  0.65   
3  2009  9601.00    8.06    40083    42911   88440    52700  187200  0.80   
4  2009   401.00    3.78    48655    49301   91224    59400  136000  0.80   

   MortDate  ...  MSA_49420  MSA_49460  MSA_49500  MSA_49620  MSA_49660  \
0      2009  ...      False      False      False      False      False   
1      2009  ...      False      False      False      False      False   
2      2009  ...      False      False      False      False      False   
3      2009  ...      False      False      False      False      False   
4      2009  ...      False      False      False      False      False   

   MSA_49700  MSA_49740  MSA_49780  MSA_49820  MSA_99999  
0      False      False    

In [3]:
df_encoded.groupby('Year').size()

Year
2009    45376
2010    33721
2011    37423
2012    56062
2013    34277
2014    29684
2015    41943
2016    55315
2017    52903
2018    59547
2019    89608
2020    82953
2021    63782
dtype: int64